In [ ]:
from pyspark.sql.functions import col,expr,count,when,regexp_extract,explode,lit,row_number,udf,collect_list,upper,regexp_replace
from pyspark.sql import SparkSession,Row,SQLContext,Window
import pyspark.sql.types as T

In [ ]:
#Load spark session
spark = SparkSession.builder \
 .master("yarn") \
 .appName("HashtagCount") \
 .config("spark.executor.instances", "2") \
 .config("spark.executor.cores", "2") \
 .config("spark.executor.memory", "1024M") \
 .getOrCreate()
sc = spark.sparkContext


In [ ]:
#Load the dataset
df = spark.read.json("hdfs:/datasets/covid")

In [ ]:
#Create a view for the dataset
df.createOrReplaceTempView("df2")

In [ ]:
#Extract all hashtags to a array of hashtags
dfV = spark.sql("""
    select  regexp_extract_all(text, '(#\\\\w+)', 1) as hashtags
    from    df2
    where   text like '%#%'
""")

In [ ]:
#Give each row an ID
dfT = dfV.withColumn("tmp",lit("ABC"))
w = Window().partitionBy('tmp').orderBy(lit('A'))
dfT = dfT.withColumn("row_id", row_number().over(w)).drop("tmp")
dfT = dfT.withColumn("hashtags",explode(dfT.hashtags))
dfT = dfT.withColumn("hashtags",upper(dfT.hashtags))


In [ ]:
#Count each time a hashtag appears
df2 = dfT.groupBy(upper(dfT.hashtags).alias("hashtags")).count()

In [ ]:
#Merge the tables of hashtags combined by their ID
dfT2 = dfT
dfT2 = dfT2.alias("a").join(
    dfT.withColumnRenamed("hashtags","H1").alias("b"),[((col("a.row_id")==col("b.row_id")) & (~(col("a.hashtags")==col("b.H1"))))],how="cross").drop(dfT.row_id)
dfT2 = dfT2.drop(dfT2.row_id).distinct()

In [ ]:
#Aggregate the hashtags in a list
dfT2 = dfT2.groupBy("hashtags").agg(collect_list("H1").alias("H1"))

In [ ]:
#Join the list of hashtags with the counter
dfT2 = dfT2.alias("a").join(df2.alias("b"),dfT2.hashtags==df2.hashtags,how="left_outer").drop(df2.hashtags)

In [ ]:
#Get all hashtags that appear at least 1000 times
dfT2 = dfT2.filter(col("count")>=1000).select("hashtags","count","H1").orderBy("hashtags")
#Replace brackets and commas with ''
dfT2 = dfT2.withColumn("H1", regexp_replace(col("H1").cast("string"),r'\[|\]|,', ''))

In [ ]:
#Write file to disk
dfT2.write.option("header",False).option("delimiter",",").csv("/user/julioferreira/task1/extra")